# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 1 ноября 2018, 06:00 
**Штраф за опоздание:** -2 балла после 06:00 1 ноября, -4 балла после 06:00 8 ноября, -6 баллов после 06:00 15 ноября

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (4 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы fit такой, чтобы она была медленнее sklearn не более чем в 10 раз. Скорость проверяем на  wine и Speed Dating Data. Для ускорения используем только numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 3 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

/home/semen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/home/semen/anaconda3/lib/python3.7/site-packages/pycodestyle.py:113: FutureWarning: Possible nested set at position 1
  EXTRANEOUS_WHITESPACE_REGEX = re.compile(r'[[({] | []}),;:]')


In [2]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
            self.G_function_aprior = self.__gini_aprior
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.G_function_aprior = self.__entropy_aprior
        elif criterion == 'misclass':
            self.G_function = self.__misclass
            self.G_function_aprior = self.__misclass_aprior
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini_aprior(self, y):
        p = np.bincount(y, minlength=self.num_class) / y.shape[0]
        return 1 - np.sum(p ** 2)

    def __entropy_aprior(self, y):
        p = (np.bincount(y, minlength=self.num_class) + 1) / y.shape[0]
        return -np.sum(p * np.log2(p))

    def __misclass_aprior(self, y):
        p = np.bincount(y, minlength=self.num_class) / y.shape[0]
        return 1 - np.max(p)

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return (l_s - np.sum(l_c ** 2, axis=1).reshape(-1, 1) / l_s) / \
            (l_s + r_s) + (r_s - np.sum(r_c ** 2, axis=1).reshape(-1, 1)
                           / r_s) / (l_s + r_s)

    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return (-np.sum(
            l_c * np.log2((l_c + 1) / (l_s + 1)), axis=1).reshape(-1, 1)
                + -np.sum(r_c * np.log2((r_c + 1) / (r_s + 1)),
                          axis=1).reshape(-1, 1) / (l_s + r_s))

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return (l_s - np.max(l_c, axis=1).reshape(-1, 1) +
                (r_s - np.max(r_c, axis=1).reshape(-1, 1))) / (l_s + r_s)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature).reshape(-1, 1)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # сортируем сэмплы по одной фиче и переставляем классы так же
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        # находим размер среза
        cut_size = np.int(self.min_samples_split / 2 - 1)

        # находим индексы, на которых происходит изменение в sorted_y (правые)
        if cut_size == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (cut_size + 1)
        # отбрасываем случай, когда таких индексов нет
        if len(r_border_ids) == 0:
            return np.inf, np.inf

        # создаем массив количеств подряд идущих классов
        eq_el_count = r_border_ids - \
            np.append(np.array([cut_size]), r_border_ids[:-1])
        # one-hote-encode для классов, строки соответствуют
        # блокам подряд идущих элементов
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        # матрица количеств идущих подряд классов,
        # по строкам подряд идущие блоки, по столбцам классы
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        # учитываем левый срез
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:cut_size], minlength=class_number)

        # для каждого порога считаем кумулятивную сумму
        # сэмплов каждого класса слева и справа
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count

        # для каждого порога считаем общее число сэмплов слева и справа
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # минимизируем неопределенность потомков
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # возвращаем оптимальное значение неопределенности
        # и значение признаков для разбиения
        left_el_id = l_sizes[idx][0]
        return gs[idx][0], (sorted_x[left_el_id-1] +
                            sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        if (depth == self.max_depth) or \
            (x.shape[0] < self.min_samples_split) or \
                ((np.bincount(y).max() / y.shape[0]) > self.sufficient_share):
            self.tree[node_id] = (self.LEAF_TYPE,
                                  np.bincount(y).argmax(),
                                  np.bincount(y, minlength=self.num_class) /
                                  y.shape[0])
            return

        features = self.get_feature_ids(x.shape[1])
        thresholds = np.array(
            [self.__find_threshold(x[:, i], y) for i in features])
        best_split_feature = thresholds[:, 0].argmin()
        impurity, threshold = thresholds[best_split_feature]
        impurity_aprior = self.G_function_aprior(y)
        left_data, right_data, left_classes, right_classes = \
            self.__div_samples(x, y, features[best_split_feature], threshold)

        if (left_data.shape[0] < self.min_samples_split) or \
                (right_data.shape[0] < self.min_samples_split):
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y, minlength=self.num_class) /
                                  y.shape[0])
            return

        # добавляем к важности фичи взвешенный Gain в этом узлеe,
        # поделим на общую длину выборки позже в fit
        self.feature_importances_[features[best_split_feature]] += \
            (impurity_aprior - impurity) * y.shape[0]

        self.tree[node_id] = (self.NON_LEAF_TYPE,
                              features[best_split_feature], threshold)
        self.__fit_node(left_data, left_classes,
                        2 * node_id + 1, depth + 1, pred_f)
        self.__fit_node(right_data, right_classes,
                        2 * node_id + 2, depth + 1, pred_f)

    def fit(self, x, y):
        self.feature_importances_ = np.zeros([x.shape[1]])
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ = self.feature_importances_ / y.shape[0]

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = \
    train_test_split(wine.data, wine.target,
                     test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

CPU times: user 2.31 ms, sys: 0 ns, total: 2.31 ms
Wall time: 2.46 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

CPU times: user 15.9 ms, sys: 0 ns, total: 15.9 ms
Wall time: 15.8 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

1.0

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

1.0

## Подготовка данных Speed Dating Data 

In [27]:
df = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')
df = df.iloc[:, :97]
df = df.drop(['id', 'idg', 'condtn', 'round', 'position', 'positin1',
              'order', 'partner', 'age_o', 'race_o', 'pf_o_att', 'pf_o_sin',
              'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o',
              'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o',
              'prob_o', 'met_o', 'field', 'undergra',
              'imprelig', 'imprace',
              'from', 'zipcode', 'career', 'sports',
              'tvsports', 'exercise',
              'dining', 'museums',
              'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv',
              'theater', 'movies', 'concerts', 'music', 'shopping',
              'yoga', 'expnum', 'wave'], axis=1)
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
pd.get_dummies(df, columns=['field_cd'], prefix='field_cd', prefix_sep='=')
pd.get_dummies(df, columns=['race'], prefix='race', prefix_sep='=')
df.loc[:, 'mn_sat'] = \
    df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] = \
    df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df.loc[:, 'income'] = \
    df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df.loc[:, 'temp_totalsum'] = \
    df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
               'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1',
           'shar1_1']] = (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                                     'fun1_1', 'amb1_1', 'shar1_1']].T /
                          df.loc[:, 'temp_totalsum'].T).T * 100
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                        'fun2_1', 'amb2_1', 'shar2_1']] \
                                   .sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1',
           'shar2_1']] = (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                     'fun2_1', 'amb2_1', 'shar2_1']].T /
                          df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    df = df.drop(feat, axis=1)

df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                .drop(['gender'], axis=1).dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                  .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                  .dropna()
df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [29]:
%time clf.fit(X_train, y_train)

CPU times: user 82 ms, sys: 7.04 ms, total: 89 ms
Wall time: 86.9 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [30]:
%time my_clf.fit(X_train, y_train)

CPU times: user 791 ms, sys: 0 ns, total: 791 ms
Wall time: 789 ms


## Проверка качества работы на Speed Dating Data

In [31]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.522809696506967

In [32]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5398272635175742

## Задание 3

In [33]:
pd.Series(data=my_clf.feature_importances_,
          index=df_pair.columns[1:]).sort_values(ascending=False).head(10)

income_f     0.008660
int_corr     0.007299
age          0.006012
attr2_1      0.004643
sinc3_1      0.004248
attr3_1      0.004123
race         0.003988
amb1_1       0.003909
tuition_f    0.003544
mn_sat_f     0.003397
dtype: float64

In [34]:
pd.Series(data=clf.feature_importances_,
           index=df_pair.columns[1:]).sort_values(ascending=False).head(10)

int_corr     0.111358
tuition_f    0.034615
amb1_1_f     0.034600
income_f     0.032560
age_f        0.028548
fun1_1_f     0.027948
sinc3_1      0.027100
sinc1_1_f    0.025233
age          0.024362
amb2_1_f     0.023688
dtype: float64

## Задание 4

In [35]:
rfc = RandomForestClassifier(n_jobs=-1, n_estimators=10)

In [36]:
params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': range(1, 10),
    'min_samples_split': range(2, 16, 5)
}

CV = GridSearchCV(estimator=rfc, param_grid=params)
CV.fit(X, y)
print(CV.best_params_)
print(CV.best_score_)

{'criterion': 'entropy', 'max_depth': 5, 'min_samples_split': 12}
0.8367091772943236
